# All

## Imports

In [1]:
# Standard Library Imports
import os
import pickle
import shutil
import time
from concurrent.futures import ThreadPoolExecutor
import os
import cv2
import asyncio
import numpy as np
from base64 import b64decode
from IPython.display import Javascript, display
import asyncio
import threading

In [2]:
!pip install fastapi
!pip install deepface
!pip install uvicorn
!pip install retinaface
!pip install colabcode
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 38.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of retinaface to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 76.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of retinaface to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install retinaface==0.0.1, retinaface==0.0.2, retinaface==

In [3]:
# Third-party Library Imports
import numpy as np
import cv2
from os import path
from tqdm import tqdm
from fastapi import FastAPI
from fastapi.responses import HTMLResponse, RedirectResponse
from retinaface import RetinaFace
from deepface import DeepFace
import deepface.commons.functions as functions
from colabcode import ColabCode
import cv2
import numpy as np
import io
import PIL
from base64 import b64decode, b64encode
from pyngrok import ngrok
from multiprocessing import Pool

Directory  /root /.deepface created
Directory  /root /.deepface/weights created


## Setting Up Weights

In [4]:
DeepFace.build_model("VGG-Face")

vgg_face_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5
To: /root/.deepface/weights/vgg_face_weights.h5
100%|██████████| 580M/580M [00:11<00:00, 49.7MB/s]


## Constants

In [5]:
FACE_MAX_WORKERS = 14
PKL_MAX_WORKERS = 12
DIRECTORY = os.getcwd()
RECORDED_FOLDER = "Recorded"
FACE_FOLDER = "Data"
PROCESSING_FOLDER = "Processing"
FACE_THRESHOLD = 0.99
FPS = 30
CAMERA_INDEX = 0
MODEL_NAME = "VGG-Face"
PKL_FOLDER = "PKL"

## Database Setup

In [6]:
folders = [FACE_FOLDER, RECORDED_FOLDER, PROCESSING_FOLDER, PKL_FOLDER]

# Create output directories if they don't exist
for folder in folders:
    if not os.path.exists(folder):
        os.makedirs(folder)

## API Setup

In [7]:
app = FastAPI()

In [8]:
cc = ColabCode(port=12000, code=False)

In [9]:
# Provide your Ngrok auth token
auth_token = "2VQ3N6dnV7bMo5PhJskNw6KOynM_43X2AJPHbCN42wdjt29jb"

# Set the auth token for Ngrok
ngrok.set_auth_token(auth_token)

## Useful Functions

In [10]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

async def take_photo_async(filename='photo.jpg', quality=0.8):
    js = Javascript('''
      async function takePhoto(quality) {
        const div = document.createElement('div');

        const video = document.createElement('video');
        video.style.display = 'block';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});

        document.body.appendChild(div);
        div.appendChild(video);
        video.srcObject = stream;
        await video.play();

        // Resize the output to fit the video element.
        google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        stream.getVideoTracks()[0].stop();
        div.remove();
        return canvas.toDataURL('image/jpeg', quality);
        }
    ''')

    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])

    print(filename)

    with open(filename, 'wb') as f:
        f.write(binary)

    return filename

In [11]:
async def capture_photos(number=100):
    tasks = []

    for i in range(number):
        filename = f"{RECORDED_FOLDER}/Frame {i}.jpg"
        task = asyncio.create_task(take_photo_async(filename=filename))
        tasks.append(task)

    await asyncio.gather(*tasks)

In [12]:
from IPython.display import Image
import os

def take_pics(number=100):
    try:
        if not os.path.exists(RECORDED_FOLDER):
            os.makedirs(RECORDED_FOLDER)

        loop = asyncio.get_event_loop()
        loop.run_until_complete(capture_photos(number))

        print("Photos captured successfully!")
    except Exception as err:
        # Handle errors
        print(str(err))


## Functionality Classes

### Face Extraction

In [13]:
# Define a class for face extraction
class FaceExtractor:
    def process_frame(self, frame, id):
        try:
            count = os.path.split(frame)[1].split(' ')[1].split('.')[0]

            # Use RetinaFace to extract faces from the frame
            faces = RetinaFace.extract_faces(frame, threshold = FACE_THRESHOLD, model = None, allow_upscaling = True)

            if not faces:
                print(f"No faces extracted in Frame {count}.")
                return None

            path0 = os.path.join(DIRECTORY, PROCESSING_FOLDER)
            if not os.path.exists(path0):
                os.makedirs(path0)

            image_sizes = [np.prod(image.shape) for image in faces]
            largest_index = np.argmax(image_sizes)
            filename2 = f"user.{id}.{count}.jpg"
            result = os.path.join(path0, filename2)
            print(f"user.{id}.{count}.jpg written")
            cv2.imwrite(result, faces[largest_index])

            return count

        except Exception as e:
            print(str(e))
            return None

    def video_to_faces_parallel(self, frames, id):
        with ThreadPoolExecutor(max_workers = FACE_MAX_WORKERS) as executor:  # Adjust max_workers as needed
            futures = [executor.submit(self.process_frame, frame, id) for frame in frames]
            for future in futures:
                future.result()  # Wait for the threads to complete

### Class Cleanup

In [14]:
class Cleanup:
    @staticmethod
    def clean():
        directory_path1 = os.path.join(DIRECTORY, RECORDED_FOLDER)
        directory_path2 = os.path.join(DIRECTORY, PROCESSING_FOLDER)
        try:
            shutil.rmtree(directory_path1)
            print(f'Directory "{directory_path1}" and its contents have been successfully deleted.')
            shutil.rmtree(directory_path2)
            print(f'Directory "{directory_path2}" and its contents have been successfully deleted.')
        except Exception as e:
            print(f'An error occurred: {e}')

    @staticmethod
    def move():
        old_fold = os.path.join(DIRECTORY, PROCESSING_FOLDER)
        new_fold = os.path.join(DIRECTORY, FACE_FOLDER)

        if not os.path.exists(FACE_FOLDER):
            os.makedirs(FACE_FOLDER)

        for filename in os.listdir(old_fold):
            old_name = os.path.join(old_fold, filename)
            new_name = os.path.join(new_fold, filename)
            if not os.path.exists(new_name):
                os.rename(old_name, new_name)
            else:
                os.remove(new_name)
                os.rename(old_name,new_name)

        print("Moved!!")

### Class Deep

In [15]:
class Deep:
    @staticmethod
    def represent(img_path, model_name = "VGG-Face", enforce_detection = True, detector_backend = "retinaface", align = True, normalization = "base"):
        resp_objs = []

        model = DeepFace.build_model(model_name)

        # ---------------------------------
        # we have run pre-process in verification. so, this can be skipped if it is coming from verify.
        target_size = functions.find_target_size(model_name = model_name)
        if detector_backend  != "skip":
            img_objs = functions.extract_faces( img = img_path, target_size = target_size, detector_backend = detector_backend, grayscale = False, enforce_detection = enforce_detection, align = align, )
        else:  # skip
            if isinstance(img_path, str):
                img = functions.load_image(img_path)
            elif type(img_path).__module__  == np.__name__:
                img = img_path.copy()
            else:
                raise ValueError(f"unexpected type for img_path - {type(img_path)}")
            # --------------------------------
            if len(img.shape)  == 4:
                img = img[0]  # e.g. (1, 224, 224, 3) to (224, 224, 3)
            if len(img.shape)  == 3:
                img = cv2.resize(img, target_size)
                img = np.expand_dims(img, axis = 0)
            # --------------------------------
            img_region = [0, 0, img.shape[1], img.shape[0]]
            img_objs = [(img, img_region, 0)]
        # ---------------------------------

        for img, region, _ in img_objs:
            # custom normalization
            img = functions.normalize_input(img = img, normalization = normalization)

            # represent
            if "keras" in str(type(model)):
                # new tf versions show progress bar and it is annoying
                embedding = model.predict(img, verbose = 0)[0].tolist()
            else:
                # SFace and Dlib are not keras models and no verbose arguments
                embedding = model.predict(img)[0].tolist()

            resp_obj = {}
            resp_obj["embedding"] = embedding
            resp_obj["facial_area"] = region
            resp_objs.append(resp_obj)

        return resp_objs

    def Create_PKL(self, model_name = "VGG-Face", distance_metric = "cosine", enforce_detection = False, detector_backend = "opencv", align = True, normalization = "base", silent = False, bid = 0000):
        tic = time.time()

        db_path = PROCESSING_FOLDER

        file_name = f"representations_{model_name}.pkl"
        file_name = file_name.replace("-", "_").lower()
        pkl = f"{PKL_FOLDER}/{bid}_{file_name}"

        # -------------------------------
        if os.path.isdir(db_path) is not True:
            raise ValueError("Passed db_path does not exist!")

        target_size = functions.find_target_size(model_name = model_name)

        # create representation.pkl from scratch
        employees = []

        for r, _, f in os.walk(db_path):
            for file in f:
                if ((".jpg" in file.lower()) or (".jpeg" in file.lower()) or (".png" in file.lower())):
                    exact_path = r + "/" + file
                    employees.append(exact_path)

        if len(employees)  == 0:
            raise ValueError("There is no image in ", db_path, " folder! Validate .jpg or .png files exist in this path.")

        # ------------------------
        # find representations for db images

        representations = []

        # for employee in employees:
        pbar = tqdm(range(0, len(employees)), desc = "Finding representations", disable = silent)
        for index in pbar:
            employee = employees[index]

            img_objs = functions.extract_faces(img = employee, target_size = target_size, detector_backend = detector_backend, grayscale = False, enforce_detection = enforce_detection, align = align)

            for img_content, _, _ in img_objs:
                embedding_obj = self.represent(img_path = img_content, model_name = model_name, enforce_detection = enforce_detection, detector_backend = "skip", align = align, normalization = normalization)

                img_representation = embedding_obj[0]["embedding"]

                instance = []
                instance.append(employee)
                instance.append(img_representation)
                representations.append(instance)

        print(pkl)

        if path.exists(pkl):
            with open(pkl, "rb") as f:
                representations_present = pickle.load(f)

                representations.extend(representations_present)

            backup_pkl = pkl.split('.')[0] + "_Backup.pkl"

            os.rename(pkl, backup_pkl)

        with open(pkl, "wb") as f:
            pickle.dump(representations, f)

        if not silent:
            print(
                f"Representations stored in {pkl} file."
                + "Please delete this file when you add new identities in your database."
            )

## API Functions

### Home Page

In [16]:
@app.get("/", response_class=HTMLResponse)
async def home():
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Prerecording</title>
    </head>
    <body>
        <div class="center-text">
            <h1>Welcome to the FastAPI API</h1>
        </div>

        <form method="GET" action="/processing_video">
            <label for="id">Enter an ID:</label>
            <input type="text" id="id" name="id" required>

            <label for="bid">Enter your Buisness ID:</label>
            <input type="text" id="bid" name="bid" required>

            <button type="submit">Let's start recording and processing!</button>
        </form>
    </body>
    </html>
    """
    return HTMLResponse(content=html_content)

### Recording Page

In [17]:
@app.get("/processing_video", response_class = RedirectResponse)
async def processing_video(id: str, bid: str):
    try:
        # Initialize the Class Object
        face_extractor = FaceExtractor()
        deep_processor = Deep()

        take_pics(number = 10)

        frames = list(os.path.join(RECORDED_FOLDER, a) for a in os.listdir(RECORDED_FOLDER))

        print(frames)

        # Process frames in parallel to extract faces
        face_extractor.video_to_faces_parallel(frames, id)

        deep_processor.Create_PKL(bid = bid)

        # Move processed frames to the data folder
        Cleanup.move()

        Cleanup.clean()

        return RedirectResponse("/postfacerep", status_code = 200)

    except Exception as e:

        return {"error": str(e)}

### Done Page

In [18]:
@app.get("/postfacerep", response_class = HTMLResponse)
async def postrep():
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>Face Extraction Done!</title>
    </head>
    <body>
        <div class = "center-text">
            <h1>Face Representation Done</h1>
        </div>
    </body>
    </html>
    """

    # Clean up recorded and processed frames

    return HTMLResponse(content = html_content)

# Calling Program Page

In [ ]:
if __name__ == "__main__":
    cc.run_app(app=app)
